# 🎵 A2SB: Ses Restorasyonu - Google Colab

**NVIDIA A2SB ile Yüksek Kaliteli Ses Restorasyonu**

Bu notebook ile ses dosyalarınızı AI ile restore edebilirsiniz!

## 🚀 Özellikler
- ✅ 44.1kHz yüksek çözünürlüklü ses restorasyonu
- ✅ Bandwidth extension (yüksek frekans tahmini)
- ✅ Audio inpainting (eksik kısımları doldurma)
- ✅ Kullanıcı dostu Gradio arayüzü

## 📋 Gereksinimler
- 🎯 **GPU**: T4 veya daha iyi (Colab Pro önerilir)
- 💾 **RAM**: Minimum 12GB
- ⏱️ **Süre**: 10 saniyelik ses için ~2-3 dakika

## 📚 Kaynaklar
- 📄 [Paper](https://arxiv.org/abs/2501.11311)
- 💻 [GitHub](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- 🤗 [Models](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

---

**Kullanım:** Hücreleri sırayla çalıştırın. Son hücre Gradio arayüzünü başlatacak!

## 📦 Adım 1: Kurulum (5-10 dakika)

Repository'yi klonlayıp gerekli kütüphaneleri yükleyelim.

In [ ]:
# Repository'yi klonla
print("📥 Repository klonlanıyor...\n")
!git clone https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git
%cd diffusion-audio-restoration-colab-Kaggle-

# Kütüphaneleri yükle
print("\n📦 Kütüphaneler yükleniyor... (5-10 dakika)\n")
!pip install -q -r requirements.txt
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q nest-asyncio

# Doğrulama
import torch
import gradio as gr
import nest_asyncio
nest_asyncio.apply()

print("\n" + "="*60)
print("✅ Kurulum tamamlandı!")
print("="*60)
print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ Gradio: {gr.__version__}")
print(f"✓ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Bellek: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print("\n🎉 Hazır!\n")

## 📥 Adım 2: Model İndirme (5-10 dakika)

İki model checkpoint'i indireceğiz (~4.5GB toplam).

In [ ]:
import os

# Checkpoint dizini oluştur
!mkdir -p ckpt

print("📥 Modeller indiriliyor... (5-10 dakika)\n")

# Model 1: One-split (0.0-1.0)
model1 = 'ckpt/A2SB_onesplit_0.0_1.0_release.ckpt'
if not os.path.exists(model1):
    print("⬇️  Model 1 indiriliyor... (~2.3GB)")
    !wget -q --show-progress https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge/resolve/main/ckpt/A2SB_onesplit_0.0_1.0_release.ckpt -O {model1}
    print("✅ Model 1 indirildi!\n")
else:
    print("✓ Model 1 zaten mevcut\n")

# Model 2: Two-split (0.5-1.0)
model2 = 'ckpt/A2SB_twosplit_0.5_1.0_release.ckpt'
if not os.path.exists(model2):
    print("⬇️  Model 2 indiriliyor... (~2.3GB)")
    !wget -q --show-progress https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge/resolve/main/ckpt/A2SB_twosplit_0.5_1.0_release.ckpt -O {model2}
    print("✅ Model 2 indirildi!\n")
else:
    print("✓ Model 2 zaten mevcut\n")

print("="*60)
print("✅ Tüm modeller hazır!")
print("="*60)

## ⚙️ Adım 3: Konfigürasyon

Model yollarını ayarlayalım.

In [ ]:
import yaml

print("⚙️  Konfigürasyon güncelleniyor...\n")

config_path = 'configs/ensemble_2split_sampling.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

config['model']['pretrained_checkpoints'] = [
    'ckpt/A2SB_onesplit_0.0_1.0_release.ckpt',
    'ckpt/A2SB_twosplit_0.5_1.0_release.ckpt'
]

with open(config_path, 'w') as f:
    yaml.dump(config, f)

print("✅ Konfigürasyon güncellendi!")
print(f"\nModel yolları:")
for i, path in enumerate(config['model']['pretrained_checkpoints'], 1):
    print(f"  {i}. {path}")

## 🎨 Adım 4: Gradio Arayüzünü Başlat

### 🚀 Ses restorasyonu için hazır!

**Nasıl kullanılır:**
1. Aşağıdaki hücreyi çalıştırın
2. Çıkan linke tıklayın (`.gradio.live` ile biter)
3. Ses dosyası yükleyin veya mikrofon ile kaydedin
4. Mod seçin:
   - **Bandwidth Extension**: Yüksek frekansları restore et (düşük kaliteli MP3'ler için)
   - **Inpainting**: Eksik kısımları doldur
5. "🚀 Restore Et" butonuna tıklayın
6. Sonucu dinleyin ve indirin!

**Ayarlar:**
- **Sampling Steps**: 25-100 (yüksek = daha iyi kalite, daha yavaş)
  - Hızlı: 25-30 adım
  - Dengeli: 50 adım ⭐ Önerilen
  - En iyi: 75-100 adım
- **Otomatik Cutoff**: Cutoff frekansını otomatik tespit et ⭐ Önerilen
- **Inpainting Uzunluğu**: 0.1-1.0 saniye

**İşlem Süreleri (T4 GPU):**
- 10 saniye ses, 50 adım: ~2-3 dakika
- 30 saniye ses, 50 adım: ~5-7 dakika
- 60 saniye ses, 50 adım: ~10-15 dakika

**⚠️ Önemli:**
- İşlem sırasında notebook'u açık tutun
- "Out of Memory" hatası alırsanız sampling steps'i azaltın
- Kısa ses dosyalarıyla test edin

In [ ]:
# Gradio arayüzünü başlat
print("🚀 Gradio arayüzü başlatılıyor...\n")
print("⏱️  Lütfen linkin görünmesini bekleyin.\n")
print("="*60)

!python gradio_app.py

print("\n" + "="*60)
print("✅ Gradio arayüzü başlatıldı!")
print("Yukarıdaki linke tıklayarak arayüze erişin.")
print("="*60)

## 💡 İpuçları ve Sorun Giderme

### ⚡ Performans

**GPU Gereksinimleri:**
- ✅ T4 (16GB) - Colab Pro
- ✅ V100 (32GB) - Colab Pro+
- ✅ A100 (40GB) - Colab Pro+
- ⚠️ Ücretsiz Colab - Bellek hatası alabilirsiniz

### 🎯 Kalite Ayarları

**Sampling Steps:**
- 25-30: Hızlı (iyi kalite)
- 50-75: Dengeli (mükemmel kalite) ⭐
- 75-100: En iyi (olağanüstü kalite)

**Cutoff Frekansı (Bandwidth Extension):**
- Otomatik tespit: Genellikle en iyi ⭐
- Manuel:
  - Düşük kalite MP3: 2000-4000 Hz
  - Orta kalite: 4000-8000 Hz
  - Yüksek kalite: 8000+ Hz

### ❌ Yaygın Hatalar

**"Out of Memory" Hatası:**
- Sampling steps'i azaltın (25-30)
- Daha kısa ses dosyası kullanın
- Colab Pro'ya geçin

**Session Timeout:**
- Colab Pro kullanın (daha uzun oturum)
- Sekmeyi aktif tutun
- Kısa ses dosyaları işleyin

### 📖 Lisans

- **Model**: NVIDIA OneWay NonCommercial License
- **Kod**: NVIDIA Source Code License - Non Commercial
- **Ticari Kullanım**: NVIDIA ile iletişime geçin
- **Araştırma**: Akademik ve araştırma amaçlı ücretsiz

### 🔗 Ek Kaynaklar

- **Paper**: [arXiv:2501.11311](https://arxiv.org/abs/2501.11311)
- **GitHub**: [test4373/diffusion-audio-restoration](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- **NVIDIA Demo**: [research.nvidia.com](https://research.nvidia.com/labs/adlr/A2SB/)
- **Models**: [HuggingFace](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

---

### 🎉 Teşekkürler!

Bu notebook'u kullandı��ınız için teşekkürler!

**Alıntı:**
```bibtex
@article{kong2025a2sb,
  title={A2SB: Audio-to-Audio Schrodinger Bridges},
  author={Kong, Zhifeng and Shih, Kevin J and Nie, Weili and Vahdat, Arash and Lee, Sang-gil and Santos, Joao Felipe and Jukic, Ante and Valle, Rafael and Catanzaro, Bryan},
  journal={arXiv preprint arXiv:2501.11311},
  year={2025}
}
```

### ⭐ Projeyi Destekle

Faydalı bulduysanız:
- ⭐ [GitHub repository](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)'ye yıldız verin
- 🐛 Hata bildirin veya özellik önerin
- 📢 Başkalarıyla paylaşın

---

**❤️ Ses restorasyonu topluluğu için yapıldı**